Prerequisites:
* `pip install numpy`
* `pip install tensorflow`
* `pip install pandas`

Let us sinchronize. Run the following code:

In [2]:
import tensorflow as tf
x = tf.placeholder(tf.float32)
sess = tf.InteractiveSession()
sess.run(x**2 - 4*x - 3, feed_dict={x:6})

9.0

The answer should be 9.0

# Neural Networks and TensorFlow crash course

Daniil Merkulov

Skoltech

## Outline:
* Meet TensorFlow (aka helloworld)
* Building simple network to solve MNIST classification problem
* TensorFlow playground
* Useful links and TensorBoard
* Optional: Another way to build TF models

## Meet TensorFlow

In [ ]:
import tensorflow as tf

1. Build the computational graph
2. Run the computational graph

### Computational graph

In [ ]:
# Run this code to obtain CG
x = tf.constant(3, dtype=tf.float32)
y = tf.constant(4.0) # also tf.float32 implicitly
sess = tf.InteractiveSession()
sess.run(x+y)

writer = tf.summary.FileWriter('C:\\Users\\brati\\Desktop\\MEGA\\Skoltech\\PhD\\TA\\Sberbank', graph=tf.get_default_graph())

Run in console
`tensorboard --logdir=PATH_DIR`

By default, the tensorboard port is `6006`, but you can setup it as you wish by adding `--port=1234`

Here is the example of computational graph:

<center><img  src="graph.png"/>

And another one, more informative: 

<center><img  src="mnist_deep.png"/>

In [ ]:
# Constants
x = tf.constant(3, dtype=tf.float32)
y = tf.constant(4.0) # also tf.float32 implicitly

In [ ]:
# Placeholders 
#SHOULD BE PROVIDED BEFORE COMPUTATIONS
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

In [ ]:
# Variables 
#SHOULD BE INITIALIZED BEFORE COMPUTATIONS
# Linear model - fully connected layer
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b
init = tf.global_variables_initializer()
sess.run(init)
sess.run(linear_model, feed_dict={x:[1]})

## MNIST classification

<center><img  src="MNIST.png"/>

55000 train, 10000 test, 5000 validation images. All of them are grayscale and 28x28

In [3]:
# Import data and libraries
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [8]:
# Create the model
h = 10
x = tf.placeholder(tf.float32, [None, 784])
W_fc1 = tf.Variable(tf.random_normal([784, h], stddev=0.1))
b_fc1 = tf.Variable(tf.random_normal([h], stddev=0.1))


h_fc1 = tf.nn.relu(tf.matmul(x, W_fc1) + b_fc1)

y = h_fc1
# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

# The raw formulation of cross-entropy,
#
#   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
#                                 reduction_indices=[1]))
#
# can be numerically unstable.
#
# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.
cross_entropy = tf.reduce_mean(
  tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)

In [9]:
# TensorFlow magic :)
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [10]:
# Train
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

# Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy is '+str(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                  y_: mnist.test.labels})))

Accuracy is 0.801


## Useful links and further reading
* [Free TensorFlow course from Google](https://www.udacity.com/course/deep-learning--ud730)
* [TensorFlow documentation](https://www.tensorflow.org/)
* [TensorFlow sandbox](playground.tensorflow.org)

## Going further. One more approach

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
COLUMNS = ["crim", "zn", "indus", "nox", "rm", "age",
           "dis", "tax", "ptratio", "medv"]
FEATURES = ["crim", "zn", "indus", "nox", "rm",
            "age", "dis", "tax", "ptratio"]
LABEL = "medv"

training_set = pd.read_csv("boston_train.csv", skipinitialspace=True,
                           skiprows=1, names=COLUMNS)
test_set = pd.read_csv("boston_test.csv", skipinitialspace=True,
                       skiprows=1, names=COLUMNS)
prediction_set = pd.read_csv("boston_predict.csv", skipinitialspace=True,
                             skiprows=1, names=COLUMNS)

In [ ]:
feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]

In [ ]:
regressor = tf.estimator.DNNRegressor(feature_columns=feature_cols,
                                      hidden_units=[10, 10],
                                      model_dir="/tmp/boston_model")

In [ ]:
def get_input_fn(data_set, num_epochs=None, shuffle=True):
  return tf.estimator.inputs.pandas_input_fn(
      x=pd.DataFrame({k: data_set[k].values for k in FEATURES}),
      y = pd.Series(data_set[LABEL].values),
      num_epochs=num_epochs,
      shuffle=shuffle)

In [ ]:
regressor.train(input_fn=get_input_fn(training_set), steps=5000)

In [ ]:
ev = regressor.evaluate(
    input_fn=get_input_fn(test_set, num_epochs=1, shuffle=False))

In [ ]:
loss_score = ev["loss"]
print("Loss: {0:f}".format(loss_score))

In [ ]:
y = regressor.predict(
    input_fn=get_input_fn(training_set, num_epochs=1, shuffle=False))
# .predict() returns an iterator of dicts; convert to a list and print
# predictions
predictions = list(p["predictions"] for p in itertools.islice(y, 6))
print("Predictions: {}".format(str(predictions)))